# [ Karate Dataset Classification with MLP ]
- train ratio : 0.1, 0.3, 0.5, 0.7
- metric : accuracy, precision, recall, F1-score
- dataset : Karate

# 1. Importing libraries & dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
ev = pd.read_csv('embedded_vector.csv')

In [3]:
data = ev[['X','Y','Color']]
data.columns = ['x1','x2','class']
data = data.sample(frac=1) # to shuffle

data.head()

,x1,x2,class
12,0.767218,0.971120,1
14,0.410940,-0.355726,0
33,-0.278933,-0.126257,0
5,0.771218,0.405933,1
23,-0.535404,0.009845,0


# 2. Define Functions

### Basic functions
- 1) train_test_split
- 2) standard scaler
- 3) transpose & matrix multiplication

In [4]:
def train_test_split(data,test_ratio):
    data.iloc[:,[0,1]] = standard_scaler(data.iloc[:,[0,1]])
    test_index = np.random.choice(len(data),int(len(data)*test_ratio),replace=False)
    train = data[~data.index.isin(test_index)]
    test = data[data.index.isin(test_index)]
    
    train_X = np.array(train)[:,[0,1]]
    train_y = np.array(train)[:,[2]].flatten()
    train_y = np.column_stack((1-train_y,train_y))
    
    test_X = np.array(test)[:,[0,1]]
    test_y = np.array(test)[:,[2]].flatten()
    test_y = np.column_stack((1-test_y,test_y))
    return train_X,train_y, test_X,test_y

In [5]:
def standard_scaler(x):
    mean = np.mean(x)
    std = np.std(x)
    return (x-mean)/std

In [6]:
def _t(X):
    return np.transpose(X)

def _m(A,B):
    return np.matmul(A,B)

### Activation functions
- 1) Sigmoid
- 2) Softmax

In [7]:
class Sigmoid:
    def __init__(self):
        self.last_o = 1
    
    def __call__(self,X):
        self.last_o = 1/(1+np.exp(-X))
        return self.last_o
    
    def grad(self):
        return self.last_o*(1-self.last_o)

In [8]:
class Softmax:
    def __init__(self):
        self.last_o = 1
        
    def __call__(self,X):
        e_x = np.exp(X-np.max(X))
        self.last_o = e_x / e_x.sum()
        return self.last_o
    
    def grad(self):
        return self.last_o*(1-self.last_o)

### Loss Function

In [9]:
class LogLoss:
    def __init__(self):
        self.dh = 1
        self.last_diff = 1
    
    def __call__(self,y,yhat):
        self.last_diff = yhat-y
        total_loss = np.mean(y*np.log(yhat+(1e-5)) + (1-y)*np.log(1-yhat+(1e-5)))
        return -total_loss
    
    def grad(self):
        return self.last_diff

# 3. Network Architecture

### 1) Neuron

In [10]:
class Neuron :
    def __init__(self,W,b,activation):
        self.W = W
        self.b = b
        self.act= activation()
        
        self.dW = np.zeros_like(self.W)  
        self.db = np.zeros_like(self.b)
        self.dh = np.zeros_like(_t(self.W)) 
        
        self.last_x = np.zeros((self.W.shape[0])) 
        self.last_h = np.zeros((self.W.shape[1]))
        
    def __call__(self,x):
        self.last_x = x
        self.last_h = _m(_t(self.W),x) + self.b
        output = self.act(self.last_h)
        return output
    
    def grad(self): 
        grad = self.act.grad()*self.W
        return grad
    
    def grad_W(self,dh): 
        grad = np.ones_like(self.W) 
        grad_a = self.act.grad()   # dh/du     
        for j in range(grad.shape[1]):
            grad[:,j] = dh[j] * grad_a[j] * self.last_x     # previous gradient * dh/du * du/dW
        return grad
        
    def grad_b(self,dh) : # dh/db = dh/du * du/db
        grad = dh * self.act.grad() * 1  # previous gradient * dh/du * du/db
        return grad

### 2) Neural Network

In [11]:
class NN:
    def __init__(self,input_num,output_num,hidden_depth,num_neuron, 
                 activation=Sigmoid, activation2=Softmax): 
        def init_var(in_,out_):
            weight = np.random.normal(0,0.1,(in_,out_))
            bias = np.zeros((out_,))
            return weight,bias
           
    ## 1-1. Hidden Layer
        self.sequence = list() # lists to put neurons
        W,b = init_var(input_num,num_neuron)
        self.sequence.append(Neuron(W,b,activation))
    
        for _ in range(hidden_depth-1):
            W,b = init_var(num_neuron,num_neuron)
            self.sequence.append(Neuron(W,b,activation)) # default : Sigmoid
    
    ## 1-2. Output Layer
        W,b = init_var(num_neuron,output_num)
        self.sequence.append(Neuron(W,b,activation2)) # default : Softmax
    
    def __call__(self,x):
        for layer in self.sequence:
            x = layer(x)
        return x
    
    def calc_grad(self,loss_fun):
        loss_fun.dh = loss_fun.grad()
        self.sequence.append(loss_fun)
        
        for i in range(len(self.sequence)-1, 0, -1):
            L1 = self.sequence[i]
            L0 = self.sequence[i-1]
            
            L0.dh = _m(L0.grad(), L1.dh)
            L0.dW = L0.grad_W(L1.dh)
            L0.db = L0.grad_b(L1.dh)
            
        self.sequence.remove(loss_fun)   

### 3) Gradient Descent

In [12]:
def GD(nn,x,y,loss_fun,lr=0.01):
    loss = loss_fun(nn(x),y) # 1) FEED FORWARD
    nn.calc_grad(loss_fun) # 2) BACK PROPAGATION
    
    for layer in nn.sequence: # Update Equation
        layer.W += -lr*layer.dW
        layer.b += -lr*layer.db    
    return loss

# 4. Train Model

In [13]:
train_X_10, train_y_10, test_X_10, test_y_10 = train_test_split(data,0.9)
train_X_30, train_y_30, test_X_30, test_y_30 = train_test_split(data,0.7)
train_X_50, train_y_50, test_X_50, test_y_50 = train_test_split(data,0.5)
train_X_70, train_y_70, test_X_70, test_y_70 = train_test_split(data,0.3)

### too Small Dataset for MLP! use simple structures!
- 1 hidden layers
- 2 neurons

### 4 cases
- case 1) train 10%
- case 2) train 30%
- case 3) train 50%
- case 4) train 70%

### case 1) train 10%

In [14]:
NeuralNet_10 = NN(2,2,1,2,activation=Sigmoid, activation2=Softmax) # input_num, output_num, hidden_depth, num_layers
loss_fun = LogLoss()
EPOCH = 16

loss_per_epoch_10 = []

for epoch in range(EPOCH):
    for i in range(train_X_10.shape[0]):
        loss = GD(NeuralNet_10,train_X_10[i],train_y_10[i],loss_fun,0.1)
    loss_per_epoch_10.append(loss)
    print('Epoch {} : Loss {}'.format(epoch+1, loss))

Epoch 1 : Loss 5.638443582537672
Epoch 2 : Loss 5.63735456325473
Epoch 3 : Loss 5.636175672761594
Epoch 4 : Loss 5.634899709608483
Epoch 5 : Loss 5.633518901764068
Epoch 6 : Loss 5.632024861641352
Epoch 7 : Loss 5.630408537596856
Epoch 8 : Loss 5.6286601616303145
Epoch 9 : Loss 5.626769192991736
Epoch 10 : Loss 5.624724257381274
Epoch 11 : Loss 5.6225130814044295
Epoch 12 : Loss 5.620122421921067
Epoch 13 : Loss 5.6175379899011855
Epoch 14 : Loss 5.614744368373833
Epoch 15 : Loss 5.611724924027651
Epoch 16 : Loss 5.608461711992781


### case 2) train 30%

In [15]:
NeuralNet_30 = NN(2,2,1,2,activation=Sigmoid, activation2=Softmax) # input_num, output_num, hidden_depth, num_layers
loss_fun = LogLoss()
EPOCH = 16

loss_per_epoch_30 = []

for epoch in range(EPOCH):
    for i in range(train_X_30.shape[0]):
        loss = GD(NeuralNet_30,train_X_30[i],train_y_30[i],loss_fun,0.1)
    loss_per_epoch_30.append(loss)
    print('Epoch {} : Loss {}'.format(epoch+1, loss))

Epoch 1 : Loss 4.61324433605692
Epoch 2 : Loss 4.483471326256113
Epoch 3 : Loss 4.328937106171388
Epoch 4 : Loss 4.14710888617625
Epoch 5 : Loss 3.9365209577209495
Epoch 6 : Loss 3.697552954573191
Epoch 7 : Loss 3.4332234653915195
Epoch 8 : Loss 3.149675279624624
Epoch 9 : Loss 2.855946346600802
Epoch 10 : Loss 2.56280022058063
Epoch 11 : Loss 2.280861273947768
Epoch 12 : Loss 2.0187589764572147
Epoch 13 : Loss 1.7820077991750098
Epoch 14 : Loss 1.5728920431585411
Epoch 15 : Loss 1.3911015017617925
Epoch 16 : Loss 1.2346514442935184


### case 3) train 50%

In [16]:
NeuralNet_50 = NN(2,2,1,2,activation=Sigmoid, activation2=Softmax) # input_num, output_num, hidden_depth, num_layers
loss_fun = LogLoss()
EPOCH = 16

loss_per_epoch_50 = []

for epoch in range(EPOCH):
    for i in range(train_X_50.shape[0]):
        loss = GD(NeuralNet_50,train_X_50[i],train_y_50[i],loss_fun,0.1)
    loss_per_epoch_50.append(loss)
    print('Epoch {} : Loss {}'.format(epoch+1, loss))

Epoch 1 : Loss 6.11396071335192
Epoch 2 : Loss 6.467335187723604
Epoch 3 : Loss 6.929678405040977
Epoch 4 : Loss 7.500208234102692
Epoch 5 : Loss 8.141667867266069
Epoch 6 : Loss 8.780571431544313
Epoch 7 : Loss 9.343024087727622
Epoch 8 : Loss 9.793079012234482
Epoch 9 : Loss 10.134170325611409
Epoch 10 : Loss 10.387771690880294
Epoch 11 : Loss 10.576856346872118
Epoch 12 : Loss 10.71980251242071
Epoch 13 : Loss 10.829848227526977
Epoch 14 : Loss 10.91618822660531
Epoch 15 : Loss 10.985170723626307
Epoch 16 : Loss 11.041213906907409


### case 4) train 70%

In [17]:
NeuralNet_70 = NN(2,2,1,2,activation=Sigmoid, activation2=Softmax) # input_num, output_num, hidden_depth, num_layers
loss_fun = LogLoss()
EPOCH = 16

loss_per_epoch_70 = []

for epoch in range(EPOCH):
    for i in range(train_X_70.shape[0]):
        loss = GD(NeuralNet_70,train_X_70[i],train_y_70[i],loss_fun,0.1)
    loss_per_epoch_70.append(loss)
    print('Epoch {} : Loss {}'.format(epoch+1, loss))

Epoch 1 : Loss 5.568678462633596
Epoch 2 : Loss 5.771129441958038
Epoch 3 : Loss 6.084428905705025
Epoch 4 : Loss 6.560187321349748
Epoch 5 : Loss 7.240610420835185
Epoch 6 : Loss 8.091164398548266
Epoch 7 : Loss 8.949737888295607
Epoch 8 : Loss 9.644239226738351
Epoch 9 : Loss 10.132579277843588
Epoch 10 : Loss 10.460603100361173
Epoch 11 : Loss 10.682989289099087
Epoch 12 : Loss 10.838346654242187
Epoch 13 : Loss 10.95063897459802
Epoch 14 : Loss 11.034442059896366
Epoch 15 : Loss 11.098772158531121
Epoch 16 : Loss 11.14936925913706


# 5. Prediction

In [18]:
def predict(model,test_X):
    preds = []
    for i in range(test_X.shape[0]):
        pred_result = np.argmax(model(test_X[i]))
        preds.append(pred_result)
    return np.array(preds)

### 1) prediction result

In [19]:
pred10 = predict(NeuralNet_10,test_X_10)

In [20]:
pred30 = predict(NeuralNet_30,test_X_30)

In [21]:
pred50 = predict(NeuralNet_50,test_X_50)

In [22]:
pred70 = predict(NeuralNet_70,test_X_70)

In [26]:
pred10

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [27]:
pred30

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0], dtype=int64)

In [28]:
pred50

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [29]:
pred70

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

### 2) metrics

In [23]:
def Metrics(pred,actual):
    TP,TN,FP,FN = 0,0,0,0
    for i in range(len(pred)):
        if pred[i]*actual[i]==1:
            TP +=1
        elif pred[i]>actual[i]:
            FP +=1
        elif pred[i]<actual[i]:
            FN +=1
        else:
            TN +=1
    
    accuracy = (TP+TN) / (TP+TN+FP+FN)
    precision = TP / (TP+FP)
    recall = TP / (TP+FN)
    F1_score = 2*(precision*recall)/(precision+recall)
    return accuracy,precision,recall,F1_score

In [24]:
print('Training Dataset 10%')
actual_class_10 = (1-test_y_10)[:,0]
Metrics(pred10,actual_class_10)

Training Dataset 10%


ZeroDivisionError: division by zero

In [ ]:
print('Training Dataset 30%')
actual_class_30 = (1-test_y_30)[:,0]
Metrics(pred30,actual_class_30)

In [ ]:
print('Training Dataset 50%')
actual_class_50 = (1-test_y_50)[:,0]
Metrics(pred50,actual_class_50)

In [ ]:
print('Training Dataset 70%')
actual_class_70 = (1-test_y_70)[:,0]
Metrics(pred70,actual_class_70)

### can check that the model is not trained well, because of the size of the dataset! ( only 34 data in total )